Importing necessary libraries

In [1]:
# -*- coding: utf-8 -*from keras.models import Sequential
import numpy as np
from numpy.random import RandomState
import torch
import scipy.io as sio
import time
# from matlab import eng

In [2]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F

Loadind stress and shape data from matlab variablle.

In [3]:
ShapeDataFile = '/content/drive/MyDrive/Colab Notebooks/Project Data/ShapeData.mat'
StressDataFile = '/content/drive/MyDrive/Colab Notebooks/Project Data/StressData.mat'
MatData_shape=sio.loadmat(ShapeDataFile)
MatData_stress=sio.loadmat(StressDataFile)

Creating the tensor of shape and stress data

In [4]:
stress_np = np.array(MatData_stress['StressData'])
shape_np = np.array(MatData_shape['ShapeData'])
# arraning the stress data for y
stress_S11 = stress_np[0,:,:]
stress_S22 = stress_np[1,:,:]
stress_S12 = stress_np[3,:,:]
stress_all = np.concatenate([stress_S11,stress_S22,stress_S12], axis=0)
# getting the tensor
shape_tensor = torch.from_numpy(shape_np)
stress_tensor = torch.from_numpy(stress_all)
dataset = TensorDataset(shape_tensor.t().float(), stress_tensor.t().float())

`A` is the predicted stress and `B` is the given stress.

`ComputeError` returns four array `MAE`, `NMAE`, `AE` and `APE` contaning respective errors for each test element.

In [5]:
#%% S11/S22/S12: 5000xN, S_all=[S11; S22; S12]
def ComputeVonMisesStress_all(S_all):

    S11_all=S_all[:, 0:5000]
    S22_all=S_all[:, 5000:10000]
    S12_all=S_all[:, 10000:15000]

    VM_all = S11_all**2 + S22_all**2 - S11_all*S22_all +3*S12_all**2
    VM_all = torch.sqrt(VM_all)

    return S11_all, S22_all, S12_all, VM_all;


In [6]:
def ComputeError(A, B):
    MAE=np.zeros(A.shape[1])
    NMAE=np.zeros(A.shape[1])
    AE=np.zeros(A.shape[1])
    APE=np.zeros(A.shape[1])
    for n in range(0, A.shape[1]):
        a=A[:,n]
        b=B[:,n]
        c=torch.absolute(a-b)
        a_abs=torch.absolute(a)
        b_abs=torch.absolute(b)
        a_max=torch.max(a_abs)
        b_max=torch.max(b_abs)
        MAE[n]=torch.mean(c)
        NMAE[n]=MAE[n]/a_max
        AE[n]=torch.absolute(a_max-b_max)
        APE[n]=AE[n]/a_max
    return MAE, NMAE, AE, APE


In [7]:
class ErrorComputer():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, train_loader, model):
        self.train_loader = train_loader
        self.MAE_S11 = []
        self.NMAE_S11 = []
        self.AE_S11 = []
        self.APE_S11 = []
        self.MAE_S22 = []
        self.NMAE_S22 = []
        self.AE_S22 = []
        self.APE_S22 =  []
        self.MAE_S12 = []
        self.NMAE_S12 = []
        self.AE_S12 = []
        self.APE_S12 = []
        self.MAE_VM = []
        self.NMAE_VM = []
        self.AE_VM = []
        self.APE_VM = []
                
    def compute(self):
        """Yield a batch of data after moving it to device"""
        for batch in self.train_loader:
            shape, stress = batch
            pred_stress = model(shape)
            [pred_S11, pred_S22, pred_S12, pred_VM] = ComputeVonMisesStress_all(pred_stress)
            [S11, S22, S12, VM] = ComputeVonMisesStress_all(stress)
            [MAE_S11, NMAE_S11, AE_S11, APE_S11] = ComputeError(pred_S11, S11)
            [MAE_S22, NMAE_S22, AE_S22, APE_S22] = ComputeError(pred_S22, S22)
            [MAE_S12, NMAE_S12, AE_S12, APE_S12] = ComputeError(pred_S12, S12)
            [MAE_VM, NMAE_VM, AE_VM, APE_VM] = ComputeError(pred_VM, VM)
            self.MAE_S11.append(MAE_S11)
            self.NMAE_S11.append(NMAE_S11)
            self.AE_S11.append(AE_S11)
            self.APE_S11.append(APE_S11)
            self.MAE_S22.append(MAE_S22)
            self.NMAE_S22.append(NMAE_S22)
            self.AE_S22.append(AE_S22)
            self.APE_S22.append(APE_S22)
            self.MAE_S12.append(MAE_S12)
            self.NMAE_S12.append(NMAE_S12)
            self.AE_S12.append(AE_S12)
            self.APE_S12.append(APE_S12)
            self.MAE_VM.append(MAE_VM)
            self.NMAE_VM.append(NMAE_VM)
            self.AE_VM.append(AE_VM)
            self.APE_VM.append(APE_VM)

    def print(self):
        print_MAE_mean = 'MAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_MAE_std = 'MAE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_NMAE_mean = 'NMAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_NMAE_std = 'NMAE std\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_AE_mean = 'AE mean\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_AE_std = 'AE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_APE_mean = 'APE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_APE_std = 'APE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print('Error\t\tS11\t\tS22\t\tS12\t\tVM')
        print('-------------------------------------------------------------------------')
        print(print_MAE_mean.format(np.mean(self.MAE_S11), np.mean(self.MAE_S22), np.mean(self.MAE_S12), np.mean(self.MAE_VM)))
        print(print_MAE_std.format(np.std(self.MAE_S11), np.std(self.MAE_S22), np.std(self.MAE_S12), np.std(self.MAE_VM)))
        print(print_NMAE_mean.format(np.mean(self.NMAE_S11), np.mean(self.NMAE_S22), np.mean(self.NMAE_S12), np.mean(self.NMAE_VM)))
        print(print_NMAE_std.format(np.std(self.NMAE_S11), np.std(self.NMAE_S22), np.std(self.NMAE_S12), np.std(self.NMAE_VM)))
        print(print_AE_mean.format(np.mean(self.AE_S11), np.mean(self.AE_S22), np.mean(self.AE_S12), np.mean(self.AE_VM)))
        print(print_AE_std.format(np.std(self.AE_S11), np.std(self.AE_S22), np.std(self.AE_S12), np.std(self.AE_VM)))
        print(print_APE_mean.format(np.mean(self.APE_S11), np.mean(self.APE_S22), np.mean(self.APE_S12), np.mean(self.APE_VM)))
        print(print_APE_std.format(np.std(self.APE_S11), np.std(self.APE_S22), np.std(self.APE_S12), np.std(self.APE_VM)))

In [8]:
"""def printError(pred_stress, stress):
    [pred_S11, pred_S22, pred_S12, pred_VM] = ComputeVonMisesStress_all(pred_stress)
    [S11, S22, S12, VM] = ComputeVonMisesStress_all(stress)
    [MAE_S11, NMAE_S11, AE_S11, APE_S11] = ComputeError(pred_S11, S11)
    [MAE_S22, NMAE_S22, AE_S22, APE_S22] = ComputeError(pred_S22, S22)
    [MAE_S12, NMAE_S12, AE_S12, APE_S12] = ComputeError(pred_S12, S12)
    [MAE_VM, NMAE_VM, AE_VM, APE_VM] = ComputeError(pred_VM, VM)
    print_MAE_mean = 'MAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print_MAE_std = 'MAE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print_NMAE_mean = 'NMAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print_NMAE_std = 'NMAE std\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print_AE_mean = 'AE mean\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print_AE_std = 'AE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print_APE_mean = 'APE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print_APE_std = 'APE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
    print('Error\t\tS11\t\tS22\t\tS12\t\tVM')
    print('-------------------------------------------------------------------------')
    print(print_MAE_mean.format(np.mean(MAE_S11), np.mean(MAE_S22), np.mean(MAE_S12), np.mean(MAE_VM)))
    print(print_MAE_std.format(np.std(MAE_S11), np.std(MAE_S22), np.std(MAE_S12), np.std(MAE_VM)))
    print(print_NMAE_mean.format(np.mean(NMAE_S11), np.mean(NMAE_S22), np.mean(NMAE_S12), np.mean(NMAE_VM)))
    print(print_NMAE_std.format(np.std(NMAE_S11), np.std(NMAE_S22), np.std(NMAE_S12), np.std(NMAE_VM)))
    print(print_AE_mean.format(np.mean(AE_S11), np.mean(AE_S22), np.mean(AE_S12), np.mean(AE_VM)))
    print(print_AE_std.format(np.std(AE_S11), np.std(AE_S22), np.std(AE_S12), np.std(AE_VM)))
    print(print_APE_mean.format(np.mean(APE_S11), np.mean(APE_S22), np.mean(APE_S12), np.mean(APE_VM)))
    print(print_APE_std.format(np.std(APE_S11), np.std(APE_S22), np.std(APE_S12), np.std(APE_VM)))"""

"def printError(pred_stress, stress):\n    [pred_S11, pred_S22, pred_S12, pred_VM] = ComputeVonMisesStress_all(pred_stress)\n    [S11, S22, S12, VM] = ComputeVonMisesStress_all(stress)\n    [MAE_S11, NMAE_S11, AE_S11, APE_S11] = ComputeError(pred_S11, S11)\n    [MAE_S22, NMAE_S22, AE_S22, APE_S22] = ComputeError(pred_S22, S22)\n    [MAE_S12, NMAE_S12, AE_S12, APE_S12] = ComputeError(pred_S12, S12)\n    [MAE_VM, NMAE_VM, AE_VM, APE_VM] = ComputeError(pred_VM, VM)\n    print_MAE_mean = 'MAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'\n    print_MAE_std = 'MAE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'\n    print_NMAE_mean = 'NMAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'\n    print_NMAE_std = 'NMAE std\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'\n    print_AE_mean = 'AE mean\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'\n    print_AE_std = 'AE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'\n    print_APE_mean = 'APE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'\n    print_APE_std = 'APE std\t\t{:.

In [9]:
class NonLinearMapping(nn.Module):
    def __init__(self, train_loader):
        super().__init__()
        self.train_loader = train_loader
        self.loss_history_batch = []
        self.loss_history = []
        self.network = nn.Sequential(
            nn.Linear(15000, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 15000))
        
    def forward(self, xb):
        return self.network(xb)
    
    def training_step(self, train_batch):
        shape, stress = train_batch
        pred_stress = self(shape)                   
        loss = F.mse_loss(pred_stress, stress)
        return loss

    def train(self, epochs, opt_func=torch.optim.Adam):
        self.optimizer = opt_func(self.parameters())

        for epoch in range(epochs):

            self.loss_history_batch = []
            
            for batch in self.train_loader:
                self.batch = batch
                self.optimizer.step(self.closure)

            self.loss_history.append(sum(self.loss_history_batch)/len(self.loss_history_batch))
            self.epoch_end(epoch,epochs, print_every=250)

    def closure(self):
        self.optimizer.zero_grad()
        loss = self.training_step(self.batch)
        self.loss_history_batch.append(loss)
        loss.backward()
        return loss
    
    def validation_step(self): 
        train_Error = ErrorComputer(self.train_loader, self)
        train_Error.compute()
        train_Error.print()
            
    def testing_step(self, test_loader):
        test_Error = ErrorComputer(test_loader, self)
        test_Error.compute()
        test_Error.print()

    def epoch_end(self, epoch, epochs, print_every=100):
        if epoch == 0 or epoch == (epochs - 1) or epoch % print_every == 0 or print_every == 'all':
            print("Epoch [{}/{}], train_loss: {:.4f}".format(epoch, epochs, self.loss_history[epoch]))

    


In [10]:
# number of times training
n = 3
for i in range(n):

    print("For epoch[{}/{}] :".format(i,n))
    t_start = time.perf_counter()

    # splitting training and testing set
    print("Splitting the dataset into training and testing data set")
    t1 = time.perf_counter()
    train_data, test_data  = random_split(dataset, [656,73])
    batch_size_train = 66
    batch_size_test = 8
    # creating batch 
    train_loader = DataLoader(train_data, batch_size_train, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_data, batch_size_test, shuffle=True, num_workers=2, pin_memory=True)
    t2 = time.perf_counter()
    print("Splitting is done. Time taken = {}".format(t2-t1))

    """creating the model and training the it"""
    # creating model
    model = NonLinearMapping(train_loader)
    # training the model
    print("Training the model : ")
    t3 = time.perf_counter()
    model.train(1000)
    t4 = time.perf_counter()
    print('time taken in trainning the model = {}'.format(t4-t3))
    # performing validation step
    t5 = time.perf_counter()
    model.validation_step()
    t6 = time.perf_counter()
    print('time taken in validatoin of final trained model with training data= {}'.format(t6-t5))
    # performing testing step
    t7 = time.perf_counter()
    model.testing_step(test_loader)
    t8 = time.perf_counter()
    print('time taken in validatoin of final trained model with testing data= {}'.format(t8-t7))

    t_end = time.perf_counter()
    print("Time taken for this epoch ={}".format(t_end-t_start))
    print("\n\n")



For epoch[0/3] :
Splitting the dataset into training and testing data set
Splitting is done. Time taken = 0.004078714999991462
Training the model : 
Epoch [0/1000], train_loss: 11546.8408
Epoch [250/1000], train_loss: 17.0300
Epoch [500/1000], train_loss: 9.9392
Epoch [750/1000], train_loss: 6.7803
Epoch [999/1000], train_loss: 8.7722
time taken in trainning the model = 1689.2532805600001
Error		S11		S22		S12		VM
-------------------------------------------------------------------------
MAE mean	2.1763		1.6337		1.2673		1.9098
MAE std		1.3984		0.8883		0.9366		1.2400
NMAE mean	0.0083		0.0127		0.0536		0.0082
NMAE std	0.0035		0.0055		0.0257		0.0033
AE mean		2.4427		2.0751		2.2110		2.2724
AE std		3.0981		3.4679		3.4168		3.1270
APE mean	0.0092		0.0157		0.0878		0.0096
APE std		0.0093		0.0253		0.1145		0.0103
time taken in validatoin of final trained model with training data= 25.38578384199991
Error		S11		S22		S12		VM
-------------------------------------------------------------------------
MAE 